# Milestone 2 — Descriptive Analysis (SQL + Python)



In [1]:
# Setup: imports + data path
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# Local data path (update if different)
data_dir = Path(r"C:\Users\Adithya\Downloads\Bibhu\SQL\athlete_events\data")
athletes_fp = data_dir / "athlete_events.csv"
noc_fp = data_dir / "noc_regions.csv"

print("Using data path:", athletes_fp)
if not athletes_fp.exists():
    raise FileNotFoundError(f"athlete_events.csv not found at {athletes_fp}. Place the CSV in this folder and re-run the notebook.")
# load CSV into pandas
df = pd.read_csv(athletes_fp, low_memory=False)
print("Loaded CSV with shape:", df.shape)
# standard helpers
df['Year'] = df['Year'].astype(int)
df['is_medal'] = df['Medal'].notna()


Using data path: C:\Users\Adithya\Downloads\Bibhu\SQL\athlete_events\data\athlete_events.csv
Loaded CSV with shape: (271116, 15)


In [2]:
# Create in-memory SQLite DB
conn = sqlite3.connect(':memory:')
df.to_sql('olympics', conn, index=False, if_exists='replace')
print('SQLite DB ready')

SQLite DB ready


## 1) Medal counts and basic distributions (SQL)

In [3]:
query = '''
SELECT Medal, COUNT(*) as count
FROM olympics
GROUP BY Medal
ORDER BY count DESC;
'''
medal_counts = pd.read_sql_query(query, conn)
medal_counts

,Medal,count
0,None,231333
1,Gold,13372
2,Bronze,13295
3,Silver,13116


## 2) Gender distribution overall and by decade (SQL + Python)

In [4]:
# gender overall
query = """
SELECT Sex, COUNT(DISTINCT ID) as athletes
FROM olympics
GROUP BY Sex;
"""
gender_overall = pd.read_sql_query(query, conn)
gender_overall

,Sex,athletes
0,F,33981
1,M,101590


In [5]:
# gender by decade via SQL
query = """
SELECT ((Year/10)*10) as Decade, Sex, COUNT(*) as records
FROM olympics
GROUP BY Decade, Sex
ORDER BY Decade;
"""
gender_by_decade = pd.read_sql_query(query, conn)
# pivot for plotting
gender_pivot = gender_by_decade.pivot(index='Decade', columns='Sex', values='records').fillna(0)
gender_pivot.head()

Sex,F,M
Decade,,
1890,0.0,380.0
1900,107.0,7964.0
1910,87.0,3953.0
1920,832.0,14727.0
1930,918.0,9804.0


## 3) Top sports & NOCs by medal-winning entries (SQL)

In [6]:
query = '''
SELECT Sport, COUNT(Medal) AS medal_count
FROM olympics
WHERE Medal IS NOT NULL
GROUP BY Sport
ORDER BY medal_count DESC
LIMIT 15;
'''
top_sports = pd.read_sql_query(query, conn)
top_sports

,Sport,medal_count
0,Athletics,3969
1,Swimming,3048
2,Rowing,2945
3,Gymnastics,2256
4,Fencing,1743
5,Football,1571
6,Ice Hockey,1530
7,Hockey,1528
8,Wrestling,1296
9,Cycling,1263


In [7]:
query = '''
SELECT NOC, COUNT(Medal) AS medal_count
FROM olympics
WHERE Medal IS NOT NULL
GROUP BY NOC
ORDER BY medal_count DESC
LIMIT 15;
'''
top_nocs = pd.read_sql_query(query, conn)
top_nocs

,NOC,medal_count
0,USA,5637
1,URS,2503
2,GER,2165
3,GBR,2068
4,FRA,1777
5,ITA,1637
6,SWE,1536
7,CAN,1352
8,AUS,1320
9,RUS,1165


## 4) Save summary CSVs (sport medals per event, noc medals per athlete) — computed in Python using SQL aggregates

In [8]:
# medals per sport (SQL) and events per sport (Python)
medals_by_sport = pd.read_sql_query("SELECT Sport, COUNT(Medal) AS MedalCount FROM olympics WHERE Medal IS NOT NULL GROUP BY Sport;", conn)
events_by_sport = df.groupby('Sport')['Event'].nunique().rename('NumEvents').reset_index()
sport_norm = medals_by_sport.merge(events_by_sport, on='Sport', how='left')
sport_norm['Medals_per_Event'] = sport_norm['MedalCount']/sport_norm['NumEvents']
# save to summary folder
summary_dir = Path.cwd() / 'milestone2_summary'
summary_dir.mkdir(exist_ok=True)
sport_norm.to_csv(summary_dir/'sport_medals_per_event.csv', index=False)
sport_norm.head()

,Sport,MedalCount,NumEvents,Medals_per_Event
0,Aeronautics,1,1,1.000000
1,Alpine Skiing,428,10,42.800000
2,Alpinism,25,1,25.000000
3,Archery,353,29,12.172414
4,Art Competitions,156,29,5.379310
